In [231]:
import requests




def get_commits(user, repo, token, per_page=100, page=1):
    url = f"https://api.github.com/repos/{user}/{repo}/commits"
    headers = {'Authorization': f'token {token}'}
    params = {'per_page': per_page, 'page': page}
    response = requests.get(url, headers=headers, params=params)
    commits = response.json()
    return commits

# Example usage
users_repos = [('Julia-Susser', 'chatGPT-impact-measurement')]
all_commits = []

for user, repo in users_repos:
    page = 1
    while True:
        commits = get_commits(user, repo, token, page=page)
        if not commits:
            break
        all_commits.extend(commits)
        page += 1

print(f'Total commits fetched: {len(all_commits)}')


Total commits fetched: 20


In [86]:
import requests
from datetime import datetime 

def search_repositories(query, token, per_page=10):
    url = "https://api.github.com/search/repositories"
    headers = {'Authorization': f'token {token}'}
    params = {'q': query, 'per_page': per_page}
    response = requests.get(url, headers=headers, params=params)
    return response.json()

def get_contributors(contributors_url, token):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(contributors_url, headers=headers)
    return response.json()

def get_user_details(user_url, token):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(user_url, headers=headers)
    return response.json()

def get_lines_of_code(repo_url, token, username):
    headers = {'Authorization': f'token {token}'}
    response = requests.get(repo_url, headers=headers)
    data = response.json()
    print(data)
    print("\n\n")
    if 'files' in data:
        for file in data['files']:
            if file['author']['login'] == username:
                return file['changes']  # or use file['additions'] + file['deletions'] for total changes
    return 0

# Example usage
query = 'university of pennsylvania "final-project"'

# Search for repositories
repositories = search_repositories(query, token)


# Collect contributors and their timestamps
users = []
for repo in repositories.get('items', []):
    contributors_url = repo['contributors_url']
    contributors = get_contributors(contributors_url, token)
    timestamp = datetime.strptime(repo['updated_at'], "%Y-%m-%dT%H:%M:%SZ")  # Convert timestamp to datetime object
    for contributor in contributors:
        print(contributor)
        users.append({'contributor': contributor["login"], 'timestamp': timestamp})
print(users)

{'login': 'HuYuxin', 'id': 6688900, 'node_id': 'MDQ6VXNlcjY2ODg5MDA=', 'avatar_url': 'https://avatars.githubusercontent.com/u/6688900?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/HuYuxin', 'html_url': 'https://github.com/HuYuxin', 'followers_url': 'https://api.github.com/users/HuYuxin/followers', 'following_url': 'https://api.github.com/users/HuYuxin/following{/other_user}', 'gists_url': 'https://api.github.com/users/HuYuxin/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/HuYuxin/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/HuYuxin/subscriptions', 'organizations_url': 'https://api.github.com/users/HuYuxin/orgs', 'repos_url': 'https://api.github.com/users/HuYuxin/repos', 'events_url': 'https://api.github.com/users/HuYuxin/events{/privacy}', 'received_events_url': 'https://api.github.com/users/HuYuxin/received_events', 'type': 'User', 'site_admin': False, 'contributions': 47}
{'login': 'akhabbaz', 'id': 11495237, 'node_id': '

In [91]:
import requests
from datetime import datetime, timedelta

def get_user_repositories(username, token):
    """ Fetch all repositories for a given user. """
    headers = {'Authorization': f'token {token}'}
    repos_url = f"https://api.github.com/users/{username}/repos"
    response = requests.get(repos_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch repositories: {response.status_code}")
        return []

def filter_repositories_by_date(repositories, timestamp, years=5):
    """ Filter repositories created within four years of a specific timestamp. """
    start_date = timestamp - timedelta(days=365*years)
    end_date = timestamp + timedelta(days=365*years)
    filtered_repos = []
    for repo in repositories:
        repo_created_at = datetime.strptime(repo['created_at'], "%Y-%m-%dT%H:%M:%SZ")
        if start_date <= repo_created_at <= end_date:
            filtered_repos.append(repo)
    print(len(filtered_repos))
    return filtered_repos

def main(username, token, timestamp):
    repositories = get_user_repositories(username, token)
    filtered_repos = filter_repositories_by_date(repositories, timestamp)


for user in users:
    username = user["contributor"]  # Change to the username you want to check
    t = user["timestamp"]
    print(t)
    main(username, token, t)


2023-01-31 08:14:22
11
2023-01-31 08:14:22
13
2023-09-08 12:13:49
0
2023-09-08 12:13:49
0
2023-09-08 12:13:49
1
2022-01-15 22:24:01
9
2022-01-15 22:24:00
9
2022-04-25 17:13:16
0
2022-04-25 17:13:16
0
2022-04-25 17:13:16
11
2016-12-01 21:00:48
5
2016-12-01 21:00:48
8
2023-11-09 23:29:24
5
2023-11-09 23:29:24
6
2015-04-23 23:33:39
16
2019-05-28 18:25:37
18
2014-08-02 17:53:22
11
2014-08-02 17:53:22
25


In [148]:
import requests

def get_trending_repositories(token):
    """Fetch a list of trending repositories on GitHub."""
    headers = {'Authorization': f'token {token}'}
    # Example URL for trending repositories; adjust query as needed
    url = "https://api.github.com/search/repositories?q=stars:>100&sort=stars&order=desc&per_page=10"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()['items']
    else:
        print(f"Failed to fetch repositories: {response.status_code}")
        return []

def get_recent_commit_users(repo, token, page):
    """Fetch recent commits from a repository and extract user data."""
    headers = {'Authorization': f'token {token}'}
    commits_url = repo['commits_url'].replace('{/sha}', '') + '?per_page=1000&page='+str(page)
    response = requests.get(commits_url, headers=headers)
    if response.status_code == 200:
        commits = response.json()
        users = {commit['author']['login'] for commit in commits if commit['author']}
        return list(users)
    else:
        print(f"Failed to fetch commits from {repo['name']}: {response.status_code}")
        return []

def main(token):
    trending_repos = get_trending_repositories(token)
    all_users = set()

    for repo in trending_repos:
        for page in range(5):
            users = get_recent_commit_users(repo, token, page)
            all_users.update(users)
            if len(all_users) >= 100:
                break

    return list(all_users)

# Example usage

users = main(token)
print(users)


['spetterman66', 'ishuah', 'lakuapik', 'maehr', 'Xom', 'rahilsh', 'rickhanlonii', 'anastasiiauk', 'abhijeetsatpute', 'fabriziocucci', 'N-E-W-T-O-N', 'kunalg12', 'saurav-2104', 'dvanbrug', 'jenna5376', 'ahmaxed', 'hoxyq', 'AtomicVar', 'lzyzsd', 'bigfootjon', 'bulhaa', 'eshellman', 'Killpit', 'ahowe442', 'riderx', 'Re1nGer', 'karoliina', 'VivekGuruduttK28', 'waleedsadek-panx', 'Aryakoste', 'invalid-email-address', 'erwanvivien', 'reiscigit', 'manuelver', 'sellisd', 'PiousSutherland', 'lerignoux', 'Any97Cris', 'ZakaryaMeddahi', 'kianmeng', 'choznerol', 'matteocrippa', 'esmaeelE', 'web-padawan', 'WOLFRIEND', 'Aegwenia', 'JoseCutileiro', 'notpresident35', 'ruishawn', 'iamnafets', 'gikf', 'n0kovo', 'keyvank', 'AgrimPrasad', 'noebrito', 'angad-singhh', 'benzoms', 'villesundell', 'cclauss', 'xulunfan', 'frustr8dlec', '0Lola', 'the-jahid', 'Ronilpatil22', 'sujaldev', 'shiyujiucsb', 'nastaran-motiee', 'nacharya-msft', 'rickstaa', 'SlightDust', 'serima', 'Shubhamkashyap1601', 'alex-semenyuk', 'Ma

In [160]:
import random as random
while True:
    user = random.choice(users)
    user= "kianmeng"
    t1 = "2022-05-30T00:00:00Z"
    t2 = "2022-11-30T00:00:00Z"
    t3 = "2024-05-30T00:00:00Z"
    get_info(user, t1, t2)
    get_info(user, t2, t3)

Lines of code contributed by user 'kianmeng' between 2022-05-30T00:00:00Z and 2022-11-30T00:00:00Z: 10 10.0
Lines of code contributed by user 'kianmeng' between 2022-05-30T00:00:00Z and 2022-11-30T00:00:00Z: 10 10.0


KeyboardInterrupt: 

In [267]:
import requests
from datetime import datetime

def get_user_public_activity(username, token, end_date_str='2022-11-30'):
    """ Check for public activity of a user up to a specified date. """
    headers = {'Authorization': f'token {token}'}
    # Set end_date to the specific past date and convert to datetime object
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    # URL to fetch user's public activity
    activity_url = f"https://api.github.com/users/{username}/events/public"
    response = requests.get(activity_url, headers=headers)

    if response.status_code == 200:
        activities = response.json()
        # Filter activities based on date, assuming activity period is one month prior to end_date
        start_date = end_date - timedelta(days=30)
        recent_activities = [act for act in activities if datetime.strptime(act['created_at'], "%Y-%m-%dT%H:%M:%SZ") > start_date]
        return len(recent_activities)
    else:
        print(f"Failed to fetch public activities for {username}: {response.status_code}")
        return None

def main(token):
    organization_members = users[:100]  # Replace with actual usernames
    for member in organization_members:
        activity_count = get_user_public_activity(member, token, '2022-11-30')
        if activity_count == 0:
            print(f"{member} has no recent public activity.")
        else:
            print(f"{member} has {activity_count} public activities in the last 30 days.")


main(token)


spetterman66 has 30 public activities in the last 30 days.
ishuah has 17 public activities in the last 30 days.
lakuapik has 29 public activities in the last 30 days.
maehr has 30 public activities in the last 30 days.
Xom has 9 public activities in the last 30 days.
rahilsh has 30 public activities in the last 30 days.
rickhanlonii has 30 public activities in the last 30 days.
anastasiiauk has 30 public activities in the last 30 days.
abhijeetsatpute has 30 public activities in the last 30 days.
fabriziocucci has 30 public activities in the last 30 days.
N-E-W-T-O-N has 30 public activities in the last 30 days.
kunalg12 has 30 public activities in the last 30 days.
saurav-2104 has no recent public activity.
dvanbrug has 13 public activities in the last 30 days.
jenna5376 has 30 public activities in the last 30 days.
ahmaxed has 30 public activities in the last 30 days.
hoxyq has 30 public activities in the last 30 days.
AtomicVar has 30 public activities in the last 30 days.
lzyzsd ha

In [310]:
import requests
from datetime import datetime



def get_lines_of_code(commits, token, username, start_date, end_date):
    total_lines_added = 0
    number_of_commits = len(commits)
    
    for commit in commits:
        try:
            commit_url = commit["url"]
            print(commit_url)
            headers = {'Authorization': f'token {token}'}
            response = requests.get(commit_url, headers=headers)
            commit_data = response.json()
            lines_added = commit_data['stats']['total']
            if lines_added > 30000:
                number_of_commits = number_of_commits-1
                continue
            total_lines_added += lines_added
            print(total_lines_added)
        except TypeError:
            continue
    
    return total_lines_added, number_of_commits


# def get_info(username_to_check, start_date, end_date):
#     lines_of_code_by_user = 0
#     num_commits = 0
    
#     l, n= get_commits(commits, token, username_to_check, start_date, end_date)
#     lines_of_code_by_user += l
#     num_commits += n
#     if lines_of_code_by_user == 0:
#         return 0
#     print(f"Lines of code contributed by user '{username_to_check}' between {start_date} and {end_date}: {lines_of_code_by_user} {lines_of_code_by_user/num_commits}")
#     return lines_of_code_by_user / num_commits

t1 = "2021-11-30T00:00:00Z"
t2 = "2022-11-30T00:00:00Z"
t3 = "2024-11-30T00:00:00Z"
# get_info("kianmeng", t1, t2)
# get_info("kianmeng", t2, t3)



In [298]:
import requests
import urllib.parse

def get_commits(username, token):
    headers = {
        'Authorization': f'token {token}',
        'Accept': 'application/vnd.github.cloak-preview'  # This header is necessary for commit search
    }
    repo_urls = set()
    page = 1
    commits = []
    while True:
        ##&since={start_date}&until={end_date}
        commit_search_url = f"https://api.github.com/search/commits?q=author:{username}&page={page}"
        print(commit_search_url)
        response = requests.get(commit_search_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            for item in data['items']:
                commits.append(item)
            # Check if there is a next page link in the headers
            if 'next' in response.links:
                page += 1
            else:
                break
        else:
            print(response)
            print(f"Failed to fetch page {page}: {response.status_code}")
            break
    return commits

# Example usage:

username = 'LarryRuane'
commits = get_commits(username, token)
start_date = '2022-05-30'  # Start of the date range
end_date = '2022-11-30'  # End of the date range
commits1 = [commit for commit in commits if commit["commit"]["committer"]["date"] > start_date and commit["commit"]["committer"]["date"] < end_date]

print(len(commits1))
start_date = '2022-11-30'  # Start of the date range
end_date = '2023-4-30'  # End of the date range
commits2 = [commit for commit in commits if commit["commit"]["committer"]["date"] > start_date and commit["commit"]["committer"]["date"] < end_date]

print(len(commits2))



https://api.github.com/search/commits?q=author:LarryRuane&page=1
https://api.github.com/search/commits?q=author:LarryRuane&page=2
https://api.github.com/search/commits?q=author:LarryRuane&page=3
https://api.github.com/search/commits?q=author:LarryRuane&page=4
https://api.github.com/search/commits?q=author:LarryRuane&page=5
https://api.github.com/search/commits?q=author:LarryRuane&page=6
https://api.github.com/search/commits?q=author:LarryRuane&page=7
https://api.github.com/search/commits?q=author:LarryRuane&page=8
https://api.github.com/search/commits?q=author:LarryRuane&page=9
https://api.github.com/search/commits?q=author:LarryRuane&page=10
https://api.github.com/search/commits?q=author:LarryRuane&page=11
https://api.github.com/search/commits?q=author:LarryRuane&page=12
https://api.github.com/search/commits?q=author:LarryRuane&page=13
https://api.github.com/search/commits?q=author:LarryRuane&page=14
https://api.github.com/search/commits?q=author:LarryRuane&page=15
https://api.github.

In [311]:
l, n= get_lines_of_code(commits1, token, username_to_check, start_date, end_date)

https://api.github.com/repos/cballou/bitcoin-opcat/commits/db929893ef0bc86ea2708cdbcf41152240cd7c73
96
https://api.github.com/repos/cballou/bitcoin-opcat/commits/c72de9990ae8f1744006d9c852023b882d5ed80c
211
https://api.github.com/repos/cballou/bitcoin-opcat/commits/48a68908ba3d5e077cda7bd1e908b923fbead824
275
https://api.github.com/repos/cballou/bitcoin-opcat/commits/bdcafb913398f0cdaff9c880618f9ebfc85c7693
277
https://api.github.com/repos/barrystyle/litedoge/commits/7a82cb54f7d068deba8a9e2b72824cc790a5d709
309
https://api.github.com/repos/barrystyle/litedoge/commits/14950691196517c0336a3ee94f295861e6f1c828
313
https://api.github.com/repos/barrystyle/litedoge/commits/c6085aad0509707b8fbaf16fa4ba7eb2585d27e0
339
https://api.github.com/repos/TKChattoraj/bitcoin/commits/db929893ef0bc86ea2708cdbcf41152240cd7c73
435
https://api.github.com/repos/TKChattoraj/bitcoin/commits/c72de9990ae8f1744006d9c852023b882d5ed80c
550
https://api.github.com/repos/TKChattoraj/bitcoin/commits/48a68908ba3d5e077c

In [312]:
l2, n2= get_lines_of_code(commits2, token, username_to_check, start_date, end_date)

https://api.github.com/repos/cballou/bitcoin-opcat/commits/72efc26439da9a1344a19569fb0cab01f82ae7d1
28
https://api.github.com/repos/cballou/bitcoin-opcat/commits/4684aa8733e831d7858d43bec4271d5d026ba183
31
https://api.github.com/repos/barrystyle/litedoge/commits/e5f7e270d67e16b27b2a41cbf7f33617b6af5df4
34
https://api.github.com/repos/barrystyle/litedoge/commits/e4276ea2716004b315779333676119a5da786207
130
https://api.github.com/repos/barrystyle/litedoge/commits/7d6caa09a363f862fcb3b4e2cce0b41b87d5bdce
245
https://api.github.com/repos/barrystyle/litedoge/commits/fd535f17fded93c305422bd54217365c54c3d643
309
https://api.github.com/repos/barrystyle/litedoge/commits/df8a5192ef3774c2922e0c36623ac8ebd26a9bee
311
https://api.github.com/repos/barrystyle/litedoge/commits/0352d7c0eab6988fc28cf7d79dcfb437eef383b8
315
https://api.github.com/repos/bitcoin-core/gui-qml/commits/72efc26439da9a1344a19569fb0cab01f82ae7d1
343
https://api.github.com/repos/bitcoin-core/gui-qml/commits/4684aa8733e831d7858d43

In [300]:
l

0

In [301]:
n

84

In [295]:
l2, n2

(7590, 29)

In [270]:
start_date = '2022-05-30'  # Start of the date range
end_date = '2022-11-30'  # End of the date range
commits2 = [commit for commit in commits if commit["commit"]["committer"]["date"] > start_date and commit["commit"]["committer"]["date"] < end_date]

print(len(commits2))


50
